In [11]:
GPT_2_CONFIGURATION_124M={
    "vocab_size":50257,
    "context_length":256,
    "emb_dim":768,
    "n_heads":12,
    "n_layers":12,
    "dropout":0.1,
    "qkv_bias":False
}

In [22]:
import math
import torch.nn as nn

In [3]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super(LayerNorm, self).__init__()
        self.eps=1e-5
        self.scale=nn.Parameter(torch.ones(emb_dim))
        self.shift=nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)
        var=x.var(dim=-1,keepdim=True)
        norm_x=(x-mean)/torch.sqrt(var+self.eps)
        return self.scale*norm_x+self.shift
    
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        # Precompute constant √(2/π) as a buffer for efficiency
        self.sqrt_2_pi = math.sqrt(2.0 / math.pi)

    def forward(self, x):
        # Ensure dtype and device compatibility
        sqrt_2_pi = torch.tensor(self.sqrt_2_pi, dtype=x.dtype, device=x.device)
        return 0.5 * x * (1 + torch.tanh(sqrt_2_pi * (x + 0.044715 * torch.pow(x, 3))))

class FeedForward(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.layers=nn.Sequential(nn.Linear(cfg["emb_dim"],4*cfg["emb_dim"]),
                                 GELU(),
                                 nn.Linear(4*cfg["emb_dim"],cfg["emb_dim"])
                                 )
        
    def forward(self,x):
        return self.layers(x)
    
    

In [4]:
class Multihead_Attention_V2(nn.Module):
    def __init__(self,d_in,d_out,context_length,num_heads,dropout,qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        self.d_out=d_out
        self.num_heads=num_heads
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_keys=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_values=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.head_dim=d_out//num_heads
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
        
    
    def forward(self,x):
        b,num_tokens,d_in=x.shape
        keys=self.W_keys(x)
        queries=self.W_query(x)
        values=self.W_values(x)
        #now change dimensions
        
        keys=keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_heads,self.head_dim)
        values=values.view(b,num_tokens,self.num_heads,self.head_dim)
        #group as num_heads
        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)
        
        attn_scores=queries @ keys.transpose(2,3)
        
        mask_bool=self.mask.bool()[:num_tokens, :num_tokens]  #if num_tokens is less than specified context length
        
        attn_scores.masked_fill_(mask_bool,-torch.inf)
        
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        attn_weights=self.dropout(attn_weights)
        
        context_vectors=(attn_weights @ values).transpose(1,2)
        context_vectors=context_vectors.contiguous().view(b,num_tokens,self.d_out)
        context_vectors=self.out_proj(context_vectors)
        
        return context_vectors
        
        
        

In [5]:
class TransformerBlock(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.att=Multihead_Attention_V2(d_in=cfg["emb_dim"],
                                       d_out=cfg["emb_dim"],
                                       context_length=cfg["context_length"],
                                       num_heads=cfg["n_heads"],
                                       dropout=cfg["dropout"],
                                       qkv_bias=cfg["qkv_bias"])
        self.ff=FeedForward(cfg)
        self.norm1=LayerNorm(cfg["emb_dim"])
        self.norm2=LayerNorm(cfg["emb_dim"])
        self.drop_shortcut=nn.Dropout(cfg["dropout"])
    
    def forward(self,x):
        shortcut=x;
        x=self.norm1(x)
        x=self.att(x)
        x=self.drop_shortcut(x)
        x=x+shortcut
        
        shortcut=x
        x=self.norm2(x)
        x=self.ff(x)
        x=self.drop_shortcut(x)
        x=x+shortcut
        return x

In [6]:
class GPT_MODEL(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb=nn.Embedding(cfg["vocab_size"],cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg["context_length"],cfg["emb_dim"])
        self.drop_emb=nn.Dropout(cfg["dropout"])
        self.trf_block=nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm=LayerNorm(cfg["emb_dim"])
        self.out_head=nn.Linear(cfg["emb_dim"],cfg["vocab_size"],bias=False)
    
    
    def forward(self,x):
        batch_size,seq_len=x.shape
        tok_embs=self.tok_emb(x)
        pos_embs=self.pos_emb(torch.arange(seq_len))
        x=tok_embs+pos_embs
        x=self.drop_emb(x)
        x=self.trf_block(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits
    
        

In [7]:
torch.manual_seed(123)
batch=torch.tensor([[6109,3626,6100,345],
                   [6109,1100,6622,257]])
model=GPT_MODEL(GPT_2_CONFIGURATION_124M)
output=model(batch)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
def generate_text(model,max_num_tokens,context_size,idx):
    for _ in range(max_num_tokens):
        idx_cond=idx[:,-context_size:]
        with torch.no_grad():
            logits=model(idx_cond)
            logits=logits[:,-1,:]
            probabs=torch.softmax(logits,dim=-1)
            idx_next=torch.argmax(probabs,dim=-1,keepdim=True)
            idx=torch.cat((idx,idx_next),dim=1)
    return idx

In [9]:
import tiktoken
def text_to_token(text,tokenizer):
    encoded=tokenizer.encode(text,allowed_special={"<|endoftext|>"})
    encoded_tensor=torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_to_text(ids,tokenizer):
    flat=ids.squeeze(0)
    text=tokenizer.decode(flat.tolist())
    return text
    
    

In [10]:
tokenizer=tiktoken.get_encoding("gpt2")
start_context="every effort moves you"
out=generate_text(model=model,max_num_tokens=10,context_size=GPT_2_CONFIGURATION_124M["context_length"],idx=text_to_token(start_context,tokenizer))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
out

tensor([[16833,  3626,  6100,   345, 36664, 48776, 14904, 15721, 25692, 41526,
          9141, 10561,  6809, 22740]])

In [12]:
text_out=token_to_text(out,tokenizer)

In [13]:
text_out

'every effort moves you compose balk overdTra Warm TOM attended manifest participants ruler'

# loss function creation

In [14]:
inputs=torch.tensor([[16833,3626,6100],
       [40,1107,588]])
targets=torch.tensor([[3626,6100,345],
                     [1107,588,11311]])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
with torch.no_grad():
    logits=model(inputs)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
logits

tensor([[[ 0.2609,  0.0452, -0.5752,  ...,  0.2083, -0.7903,  0.4520],
         [-0.7940, -0.4352, -0.9753,  ...,  0.6932, -0.9638, -0.3656],
         [ 0.8137, -0.3918, -0.2658,  ...,  0.7349, -0.4702, -0.1247]],

        [[-0.4497,  0.1390, -0.2266,  ..., -0.3854,  0.7435, -0.0967],
         [-0.9131, -0.0038, -0.6072,  ...,  0.3176,  0.0071, -0.1930],
         [ 0.6040,  0.5497,  0.4188,  ..., -0.7931,  1.1989, -0.3967]]])

In [18]:
probabs=torch.softmax(logits,dim=-1)

<IPython.core.display.Javascript object>

In [19]:
probabs

tensor([[[2.1924e-05, 1.7670e-05, 9.5021e-06,  ..., 2.0801e-05,
          7.6628e-06, 2.6541e-05],
         [7.6336e-06, 1.0929e-05, 6.3678e-06,  ..., 3.3779e-05,
          6.4416e-06, 1.1716e-05],
         [3.8089e-05, 1.1409e-05, 1.2941e-05,  ..., 3.5203e-05,
          1.0548e-05, 1.4902e-05]],

        [[1.0771e-05, 1.9404e-05, 1.3462e-05,  ..., 1.1486e-05,
          3.5516e-05, 1.5330e-05],
         [6.7415e-06, 1.6736e-05, 9.1541e-06,  ..., 2.3081e-05,
          1.6921e-05, 1.3851e-05],
         [3.0778e-05, 2.9150e-05, 2.5574e-05,  ..., 7.6120e-06,
          5.5797e-05, 1.1315e-05]]])

In [20]:
token_ids=torch.argmax(probabs,dim=-1,keepdim=True)

<IPython.core.display.Javascript object>

In [21]:
token_ids

tensor([[[19146],
         [34586],
         [26729]],

        [[ 9580],
         [25537],
         [41751]]])

In [23]:
text_idx=0
target_probabs1=probabs[text_idx,[0,1,2],targets[text_idx]]
text_idx=1
target_probabs2=probabs[text_idx,[0,1,2],targets[text_idx]]

In [24]:
target_probabs1

tensor([8.9136e-05, 4.1332e-05, 1.8305e-05])

In [25]:
target_probabs2

tensor([1.0141e-05, 6.0014e-05, 4.4526e-06])

In [26]:
log_probabs=torch.log(torch.cat((target_probabs1,target_probabs2)))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
log_probabs

tensor([ -9.3253, -10.0939, -10.9083, -11.4989,  -9.7209, -12.3220])

In [28]:
avg_log_probab=torch.mean(log_probabs)

<IPython.core.display.Javascript object>

In [30]:
avg_log_probab

tensor(-10.6449)

In [31]:
neg_avg_log_probab=-1*avg_log_probab

# method2 using direct function

In [33]:
logits_flat=logits.flatten(0,1)
targets_flat=targets.flatten()

In [35]:
logits_flat

tensor([[ 0.2609,  0.0452, -0.5752,  ...,  0.2083, -0.7903,  0.4520],
        [-0.7940, -0.4352, -0.9753,  ...,  0.6932, -0.9638, -0.3656],
        [ 0.8137, -0.3918, -0.2658,  ...,  0.7349, -0.4702, -0.1247],
        [-0.4497,  0.1390, -0.2266,  ..., -0.3854,  0.7435, -0.0967],
        [-0.9131, -0.0038, -0.6072,  ...,  0.3176,  0.0071, -0.1930],
        [ 0.6040,  0.5497,  0.4188,  ..., -0.7931,  1.1989, -0.3967]])

In [36]:
targets_flat

tensor([ 3626,  6100,   345,  1107,   588, 11311])

In [38]:
loss=torch.nn.functional.cross_entropy(logits_flat,targets_flat)

<IPython.core.display.Javascript object>

In [39]:
loss

tensor(10.6449)

# calculation loss on actual dataset

In [1]:
with open("the-verdict.txt","r") as file:
    raw_text=file.read()

In [3]:
import tiktoken

In [4]:
tokenizer=tiktoken.get_encoding("gpt2")
encoded=tokenizer.encode(raw_text)

In [16]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTdataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # Correct allowed_special to be a set
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Correct slicing for target_chunk
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            
            # Convert to torch tensors
            self.input_ids.append(torch.tensor(input_chunk, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunk, dtype=torch.long))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [17]:
def dataloader(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
    tokenizer=tiktoken.get_encoding("gpt2")
    dataset=GPTdataset(txt,tokenizer,max_length,stride)
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
    return dataloader
    

In [18]:
train_ratio=0.9
split_idx=int(train_ratio*len(raw_text))
train_data=raw_text[:split_idx]
val_data=raw_text[split_idx:]
torch.manual_seed(123)
train_loader=dataloader(train_data,batch_size=2,max_length=GPT_2_CONFIGURATION_124M["context_length"],stride=GPT_2_CONFIGURATION_124M["context_length"],drop_last=True,shuffle=True,num_workers=0)
val_loader=dataloader(val_data,batch_size=2,max_length=GPT_2_CONFIGURATION_124M["context_length"],stride=GPT_2_CONFIGURATION_124M["context_length"],drop_last=False,shuffle=False,num_workers=0)

In [20]:
for x,y in train_loader:
    print(x.shape)

torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])
torch.Size([2, 256])


In [35]:
class GPT_MODEL(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb=nn.Embedding(cfg["vocab_size"],cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg["context_length"],cfg["emb_dim"])
        self.drop_emb=nn.Dropout(cfg["dropout"])
        self.trf_block=nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm=LayerNorm(cfg["emb_dim"])
        self.out_head=nn.Linear(cfg["emb_dim"],cfg["vocab_size"],bias=False)
    
    
    def forward(self,x):
        batch_size,seq_len=x.shape
        tok_embs=self.tok_emb(x)
        pos_embs=self.pos_emb(torch.arange(seq_len,device=x.device))
        x=tok_embs+pos_embs
        x=self.drop_emb(x)
        x=self.trf_block(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits
    
        

In [36]:
class TransformerBlock(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.att=Multihead_Attention_V2(d_in=cfg["emb_dim"],
                                       d_out=cfg["emb_dim"],
                                       context_length=cfg["context_length"],
                                       num_heads=cfg["n_heads"],
                                       dropout=cfg["dropout"],
                                       qkv_bias=cfg["qkv_bias"])
        self.ff=FeedForward(cfg)
        self.norm1=LayerNorm(cfg["emb_dim"])
        self.norm2=LayerNorm(cfg["emb_dim"])
        self.drop_shortcut=nn.Dropout(cfg["dropout"])
    
    def forward(self,x):
        shortcut=x;
        x=self.norm1(x)
        x=self.att(x)
        x=self.drop_shortcut(x)
        x=x+shortcut
        
        shortcut=x
        x=self.norm2(x)
        x=self.ff(x)
        x=self.drop_shortcut(x)
        x=x+shortcut
        return x

In [37]:
class Multihead_Attention_V2(nn.Module):
    def __init__(self,d_in,d_out,context_length,num_heads,dropout,qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        self.d_out=d_out
        self.num_heads=num_heads
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_keys=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_values=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.head_dim=d_out//num_heads
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
        
    
    def forward(self,x):
        b,num_tokens,d_in=x.shape
        keys=self.W_keys(x)
        queries=self.W_query(x)
        values=self.W_values(x)
        #now change dimensions
        
        keys=keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_heads,self.head_dim)
        values=values.view(b,num_tokens,self.num_heads,self.head_dim)
        #group as num_heads
        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)
        
        attn_scores=queries @ keys.transpose(2,3)
        
        mask_bool=self.mask.bool()[:num_tokens, :num_tokens]  #if num_tokens is less than specified context length
        
        attn_scores.masked_fill_(mask_bool,-torch.inf)
        
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
        attn_weights=self.dropout(attn_weights)
        
        context_vectors=(attn_weights @ values).transpose(1,2)
        context_vectors=context_vectors.contiguous().view(b,num_tokens,self.d_out)
        context_vectors=self.out_proj(context_vectors)
        
        return context_vectors
        

In [38]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super(LayerNorm, self).__init__()
        self.eps=1e-5
        self.scale=nn.Parameter(torch.ones(emb_dim))
        self.shift=nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)
        var=x.var(dim=-1,keepdim=True)
        norm_x=(x-mean)/torch.sqrt(var+self.eps)
        return self.scale*norm_x+self.shift
    
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        # Precompute constant √(2/π) as a buffer for efficiency
        self.sqrt_2_pi = math.sqrt(2.0 / math.pi)

    def forward(self, x):
        # Ensure dtype and device compatibility
        sqrt_2_pi = torch.tensor(self.sqrt_2_pi, dtype=x.dtype, device=x.device)
        return 0.5 * x * (1 + torch.tanh(sqrt_2_pi * (x + 0.044715 * torch.pow(x, 3))))

class FeedForward(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.layers=nn.Sequential(nn.Linear(cfg["emb_dim"],4*cfg["emb_dim"]),
                                 GELU(),
                                 nn.Linear(4*cfg["emb_dim"],cfg["emb_dim"])
                                 )
        
    def forward(self,x):
        return self.layers(x)
    
    

In [39]:
torch.manual_seed(123)
model=GPT_MODEL(GPT_2_CONFIGURATION_124M)
model.eval()

GPT_MODEL(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_block): Sequential(
    (0): TransformerBlock(
      (att): Multihead_Attention_V2(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_keys): Linear(in_features=768, out_features=768, bias=False)
        (W_values): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): Multihead_Attention_V2(
        (W_query): Linear(i

In [52]:
def loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches
        
    

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# However, the resulting loss values may be slightly different.

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")
#
# print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = loss_loader(train_loader, model, device)
    val_loss = loss_loader(val_loader, model, device)

    print("Training loss:", train_loss)
    print("Validation loss:", val_loss)

Training loss: 10.98738521999783
Validation loss: 10.980905532836914
